In [16]:
import os
from collections import defaultdict
from itertools import combinations
import numpy as np
from scipy.sparse import csc_array, csc_matrix
from scipy.sparse.linalg import eigsh
import jax
import jax.numpy as jnp
from heavyhex_qft.triangular_z2 import TriangularZ2Lattice
from pymatching import Matching

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
jax.config.update('jax_enable_x64', True)

In [ ]:
lattice = TriangularZ2Lattice('''
 * * *
* * * *
 * * *
''')

## Charge sectors

In [3]:
nv = lattice.num_vertices
sectors = {}
for nq in range(0, nv + 2, 2):
    for vids in combinations(range(nv), nq):
        charge_config = np.zeros(nv, dtype=np.uint8)
        charge_config[::-1][list(vids)] = 1
        sectors[vids] = lattice.charge_subspace(charge_config)

In [ ]:
subspace_dims = set(sector.shape[0] for sector in sectors.values())
if len(subspace_dims) != 1:
    raise RuntimeError('Subspaces should all have the same dimensionality')
subdim = subspace_dims.pop()
if subdim != 2 ** lattice.num_plaquettes:
    raise RuntimeError('Subspace dimension is not 2^np')
print('Charge sector dimension:', subdim)
sizes = defaultdict(int)
for vids in sectors:
    sizes[len(vids)] += 1
print('Sector sizes:')
for nq in sorted(sizes.keys()):
    print(f'Q={nq}', sizes[nq])

Charge sector dimension: 1024
Sector sizes:
Q=0 1
Q=2 45
Q=4 210
Q=6 210
Q=8 45
Q=10 1


## Verify block diagonality of the Hamiltonian

In [5]:
hmat = csc_array(lattice.make_hamiltonian(1.).to_matrix(sparse=True))

In [6]:
for vids, subspace in sectors.items():
    ind_starts = hmat.indptr[subspace]
    ind_ends = hmat.indptr[subspace + 1]
    rows = set()
    for start, end in zip(ind_starts, ind_ends):
        rows |= set(hmat.indices[start:end].tolist())
    if rows != set(subspace.tolist()):
        raise RuntimeError(f'Sector {vids} is mapped out by H')

## Diagonalize each block

In [ ]:
hblocks = np.zeros((len(sectors), subdim, subdim), dtype=np.complex128)
for isect, subspace in enumerate(sectors.values()):
    ind_starts = hmat.indptr[subspace]
    ind_ends = hmat.indptr[subspace + 1]
    for icol, (start, end) in enumerate(zip(ind_starts, ind_ends)):
        rows = hmat.indices[start:end]
        irows = np.searchsorted(subspace, rows)
        hblocks[isect, irows, icol] = hmat.data[start:end]
    
eigvals = jnp.linalg.eigh(hblocks)[0]

In [8]:
spectra = {vids: eigvals[isect] for isect, vids in enumerate(sectors.keys())}

## Construct a dual Lattice for each sector

In [17]:
nv = lattice.num_vertices
nl = lattice.num_links
matching_matrix = np.zeros((nv, nl), dtype=int)
for iv in range(lattice.num_vertices):
    matching_matrix[iv, lattice.vertex_links(iv)] = 1
matching = Matching(csc_matrix(matching_matrix[::-1, ::-1]))

In [27]:
dual_lattices = {}
for vids in sectors:
    syndrome = np.zeros(nv, dtype=int)
    syndrome[nv - 1 - np.array(vids, dtype=int)] = 1
    base_link_state = matching.decode(syndrome)
    dual_lattices[vids] = lattice.plaquette_dual(base_link_state=base_link_state)

## Diagonalize dual Hamiltonians

In [31]:
hduals = np.zeros_like(hblocks)
for isect, dual_lattice in enumerate(dual_lattices.values()):
    hduals[isect] = dual_lattice.make_hamiltonian(1.).to_matrix(sparse=False)
eigvals = jnp.linalg.eigh(hduals)[0]

In [33]:
dual_spectra = {vids: eigvals[isect] for isect, vids in enumerate(sectors.keys())}

In [37]:
for vids, spectrum in spectra.items():
    if not np.allclose(spectrum, dual_spectra[vids]):
        raise RuntimeError(f'Spectra of primal block and dual do not match for {vids}')